In [2]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

In [3]:
res = requests.get('https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/sick2014/SICK_train.txt')

text = res.text
text[:100]

'pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment\n1\tA group of kids is playing in '

In [31]:
lines= text.split("\n")
lines[:5]

['pair_ID\tsentence_A\tsentence_B\trelatedness_score\tentailment_judgment',
 '1\tA group of kids is playing in a yard and an old man is standing in the background\tA group of boys in a yard is playing and a man is standing in the background\t4.5\tNEUTRAL',
 '2\tA group of children is playing in the house and there is no man standing in the background\tA group of kids is playing in a yard and an old man is standing in the background\t3.2\tNEUTRAL',
 '3\tThe young boys are playing outdoors and the man is smiling nearby\tThe kids are playing outdoors near a man with a smile\t4.7\tENTAILMENT',
 '5\tThe kids are playing outdoors near a man with a smile\tA group of kids is playing in a yard and an old man is standing in the background\t3.4\tNEUTRAL']

In [32]:
data = pd.read_csv(StringIO(text), sep='\t')
data.head()

,pair_ID,sentence_A,sentence_B,relatedness_score,entailment_judgment
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL


In [33]:
sentences = data['sentence_A'].tolist()
sentences[:5]

['A group of kids is playing in a yard and an old man is standing in the background',
 'A group of children is playing in the house and there is no man standing in the background',
 'The young boys are playing outdoors and the man is smiling nearby',
 'The kids are playing outdoors near a man with a smile',
 'The young boys are playing outdoors and the man is smiling nearby']

And we will pull in the sentence_B column too, giving us ~4.5K unique sentences.

In [34]:
sentence_b = data['sentence_B'].tolist()
sentences.extend(sentence_b)
len(set(sentences))

4802

This isn't a particularly large number, so let's pull in a few more similar datasets.



In [35]:
urls = [
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.train.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/MSRpar.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2012/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2013/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/OnWN.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2014/images.test.tsv',
    'https://raw.githubusercontent.com/brmson/dataset-sts/master/data/sts/semeval-sts/2015/images.test.tsv'
]

In [49]:
for url in urls:
    res = requests.get(url)
    # extract to dataframe
    data = pd.read_csv(StringIO(res.text), sep='\t',header=None, on_bad_lines="skip")
    # add to columns 1 and 2 to sentences list
    sentences.extend(data[1].tolist())
    sentences.extend(data[2].tolist())

In [50]:
len(set(sentences))


14505

Before converting to our sentence embeddings, we will save to text file as backup.



In [63]:
# remove duplicates and NaN
sentences = [
    sentence.replace('\n', '') for sentence in list(set(sentences)) if type(sentence) is str
    ]

In [64]:
len(sentences)

14504

In [65]:
with open('sentences.txt', 'w') as fp:
    fp.write('\n'.join(sentences))

Now we have 14.5K unique sentences, a much better size. We'll go ahead and build the sentence embeddings (this can take some time, feel free to download the embeddings from here). TODO add link



In [68]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

c:\Users\ASUS\Desktop\bsoft\ai-list\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\Desktop\bsoft\ai-list\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--sentence-transformers--bert-base-nli-mean-tokens. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an adm

(14504, 768)

We can save/load from file in the case of needing to reload the notebook for any reason later.



In [71]:
sentence_embeddings.shape[0]

14504

In [76]:
with open(f'./sim_sentences/embeddings_X.npy', 'wb') as fp:
    np.save(fp, sentence_embeddings[0:256])

In [ ]:
# saving data
split = 256
file_count = 0
for i in range(0, sentence_embeddings.shape[0], split):
    end = i + split
    if end > sentence_embeddings.shape[0] + 1:
        end = sentence_embeddings.shape[0] + 1
    file_count = '0' + str(file_count) if file_count < 0 else str(file_count)
    with open(f'./sim_sentences/embeddings_{file_count}.npy', 'wb') as fp:
        np.save(fp, sentence_embeddings[i:end, :])
    print(f"embeddings_{file_count}.npy | {i} -> {end}")
    file_count = int(file_count) + 1

embeddings_0.npy | 0 -> 256
embeddings_1.npy | 256 -> 512
embeddings_2.npy | 512 -> 768
embeddings_3.npy | 768 -> 1024
embeddings_4.npy | 1024 -> 1280
embeddings_5.npy | 1280 -> 1536
embeddings_6.npy | 1536 -> 1792
embeddings_7.npy | 1792 -> 2048
embeddings_8.npy | 2048 -> 2304
embeddings_9.npy | 2304 -> 2560
embeddings_10.npy | 2560 -> 2816
embeddings_11.npy | 2816 -> 3072
embeddings_12.npy | 3072 -> 3328
embeddings_13.npy | 3328 -> 3584
embeddings_14.npy | 3584 -> 3840
embeddings_15.npy | 3840 -> 4096
embeddings_16.npy | 4096 -> 4352
embeddings_17.npy | 4352 -> 4608
embeddings_18.npy | 4608 -> 4864
embeddings_19.npy | 4864 -> 5120
embeddings_20.npy | 5120 -> 5376
embeddings_21.npy | 5376 -> 5632
embeddings_22.npy | 5632 -> 5888
embeddings_23.npy | 5888 -> 6144
embeddings_24.npy | 6144 -> 6400
embeddings_25.npy | 6400 -> 6656
embeddings_26.npy | 6656 -> 6912
embeddings_27.npy | 6912 -> 7168
embeddings_28.npy | 7168 -> 7424
embeddings_29.npy | 7424 -> 7680
embeddings_30.npy | 7680 -> 7

In [87]:
d = sentence_embeddings.shape[1]

In [88]:
d

768

## Flat L2 Index
We initialize the flat L2 distance index IndexFlatL2, all we need is the specify the vector dimensionality - which in this case is d == 768 (to align with the sentence-BERT model output embeddings of size 768).

In [89]:
index = faiss.IndexFlatL2(d)


Often, we will use indexes that require us to train them on our data before being used (if we are grouping or transforming the data in any way). IndexFlatL2 however, is a simple operation and only requires that we calculate distances between vectors when we introduce our query vector xq during search. So, in this case, no training is required - which we can confirm by checking the is_trained attribute.



In [90]:
index.is_trained


True

Okay so once we're happy that our index is prepared, we then add new vectors using the add method.



In [91]:
index.ntotal

0

In [92]:
index.add(sentence_embeddings)

In [93]:
index.ntotal

14504

Then search given a query xq and number of nearest neigbors to return k.



In [94]:
k = 4
xq = model.encode(["Someone sprints with a football"])

In [96]:
xq.shape

(1, 768)

In [103]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[ 9944  8950  3193 10612]]
CPU times: total: 15.6 ms
Wall time: 7.72 ms


In [104]:
[f'{i}: {sentences[i]}' for i in I[0]]


['9944: A group of football players is running in the field',
 '8950: A group of people playing football is running in the field',
 '3193: Two groups of people are playing football',
 '10612: A person playing football is running past an official carrying a football']

In [105]:
sentences[7460]

'The camera man is nailing a hammer to the wall'

Clearly we have some good matches, everything returned includes people running with a football, or on the context of a football match. Now, if we'd rather extract the numerical vectors from FAISS, we can do that too.

In [107]:
vecs = np.zeros((k, d))
for i, val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [110]:
vecs.shape


(4, 768)

In [114]:
vecs[0][:100]


array([ 0.01627032,  0.22325929, -0.1503737 , -0.30747244, -0.27122408,
       -0.10593212, -0.06460895,  0.04738218, -0.73349065, -0.37657702,
       -0.76762843,  0.16902918,  0.53107667,  0.51176614,  1.14415848,
       -0.08562871, -0.67240077, -0.96637088,  0.02545463, -0.21559834,
       -1.25656617, -0.829822  , -0.09824976, -0.21850872,  0.50610274,
        0.10527936,  0.50396872,  0.65242982, -1.39458692,  0.65847498,
       -0.21525335, -0.22487463,  0.81818342,  0.08464301, -0.76141685,
       -0.28928319, -0.09825782, -0.73046154,  0.07855817, -0.84354562,
       -0.59242058,  0.77471381, -1.20920551, -0.22757973, -1.30733621,
       -0.23081462, -1.31322527,  0.01629065, -0.97285461,  0.19308166,
        0.47424555,  1.18920863, -1.96741259, -0.70061117, -0.29638755,
        0.6053372 ,  0.62407428, -0.70340371, -0.86754197,  0.17673095,
       -0.19170596, -0.02951976,  0.22623527, -0.16695422, -0.80402517,
       -0.45918941,  0.69675511, -0.24928217, -1.01478708, -0.92

## Adding Partitioning to the Index
FAISS allows us to add an additional step to optimize our search efficiency using a variety of different methods. A popular approach is to partition the index into Voronoi cells. Using this method we would take our query vector xq, identify the cell it belongs to, and then use our IndexFlatL2 to search between the query vector xq and all indexed vectors belonging to that cell. We can also include vectors from other nearby cells too.

We initialize our new partitioned index by first adding our previous IndexFlatL2 operation as a quantization step (another step in the search process), and feeding this into the new IndexIVFFlat operation like so:

In [115]:
nlist = 50
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

Here we've added a new parameter nlist. We use nlist to define how many partitions we'd like our index to have.

When we built the previous, IndexFlatL2-only index, we noted that no training was required as no grouping/transformation was required to build that index. Now that we've added partitioning using IndexIVFFlat, this is no longer the case. Let's take a look at the is_trained attribute.

In [116]:
index.is_trained


False

So, what we need to do now is train our index on our data, which we do before adding any data to the index (otherwise the index cannot know how to group each vector).



In [117]:
index.train(sentence_embeddings)
index.is_trained

True

Now our index is trained, we add our data just as we did before.



In [118]:
index.ntotal

0

In [119]:
index.add(sentence_embeddings)
index.ntotal

14504

Let's search again using the same indexed sentence embeddings and the same query xq.



In [121]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 3193 10612  2711   698]]
CPU times: total: 0 ns
Wall time: 632 μs


We can increase the number of nearby cells to search too with nprobe.



In [122]:
index.nprobe = 10


In [125]:
%%time
D, I = index.search(xq, k)  # search
print(I)

[[ 9944  8950  3193 10612]]
CPU times: total: 0 ns
Wall time: 1.35 ms


Increasing the number of nprobe will improve the accuracy of our search, but cost time. Our earlier IndexFlatL2-only search was exhaustive (it compared every single vector) and so it identified the closest matches with a perfect accuracy. The smaller our nprobe value, the smaller scope that we search. We received perfect results (that matched our previous IndexFlatL2-only results - 7460, 10940, 3781, 5747), however, if we found that we were not getting closely matching results, we could simply bump nprobe up further - improving accuracy, but increasing time-taken too.

It's worth noting that the time taken can change with each run too, if we rerun the above block, we usually get a different time:

In [133]:
%%time
D, I = index.search(xq,k)
print(I)

[[ 9944  8950  3193 10612]]
CPU times: total: 0 ns
Wall time: 1.03 ms


For IVF (and IMI) indexes, before attempting to use the reconstruct method, we need to call the make_direct_map method - otherwise we will return a RunetimeError.

In [135]:
index.reconstruct(9944)


RuntimeError: Error in idx_t __cdecl faiss::DirectMap::get(idx_t) const at D:\a\faiss-wheels\faiss-wheels\faiss\faiss\invlists\DirectMap.cpp:83: direct map not initialized

In [136]:
index.make_direct_map()


In [143]:
index.reconstruct(9944)[:100]

array([ 0.01627032,  0.22325929, -0.1503737 , -0.30747244, -0.27122408,
       -0.10593212, -0.06460895,  0.04738218, -0.73349065, -0.37657702,
       -0.76762843,  0.16902918,  0.53107667,  0.51176614,  1.1441585 ,
       -0.08562871, -0.6724008 , -0.9663709 ,  0.02545463, -0.21559834,
       -1.2565662 , -0.829822  , -0.09824976, -0.21850872,  0.50610274,
        0.10527936,  0.5039687 ,  0.6524298 , -1.3945869 ,  0.658475  ,
       -0.21525335, -0.22487463,  0.8181834 ,  0.08464301, -0.76141685,
       -0.2892832 , -0.09825782, -0.73046154,  0.07855817, -0.8435456 ,
       -0.5924206 ,  0.7747138 , -1.2092055 , -0.22757973, -1.3073362 ,
       -0.23081462, -1.3132253 ,  0.01629065, -0.9728546 ,  0.19308166,
        0.47424555,  1.1892086 , -1.9674126 , -0.7006112 , -0.29638755,
        0.6053372 ,  0.6240743 , -0.7034037 , -0.86754197,  0.17673095,
       -0.19170596, -0.02951976,  0.22623527, -0.16695422, -0.8040252 ,
       -0.45918941,  0.6967551 , -0.24928217, -1.0147871 , -0.92

## Quantization
Well, when storing these vectors we're storing the full (eg Flat) vector. Now in very big datasets this can quickly become a problem. Typically, we look at big datasets, and when working with large dataset we will find that storing the full vectors consumes too much space.

Fortunately, FAISS comes with the ability to compress our vectors using transformations based on Product Quantization (PQ). But, what is PQ? Well, we can view it as an additional approximation step similar to our use of IVF, which allowed us to approximate by reducing the scope of our search. PQ is slightly different however, and approximates the distance (or similarity) calculation instead.

In [144]:
m = 8  # number of centroid IDs in final compressed vectors
bits = 8 # number of bits in each centroid

quantizer = faiss.IndexFlatL2(d)  # we keep the same L2 distance flat index
index = faiss.IndexIVFPQ(quantizer, d, nlist, m, bits) 

In [145]:
index.is_trained


False

In [146]:
index.train(sentence_embeddings)


In [147]:
index.add(sentence_embeddings)


In [148]:
index.nprobe

1

Let's compare it to our previous index without PQ, and an nprobe value of 10.



In [149]:
index.nprobe = 10


In [155]:
%%time
D, I = index.search(xq,k)
print(I)

[[2332 2711 2902 3193]]
CPU times: total: 0 ns
Wall time: 679 μs


Through adding PQ we've reduced our search time from ~7.5ms to ~5ms, a small difference on a dataset of this size, but when scaled to larger datasets this can make a huge difference.

Now, we should also notice the slightly different results being returned. Beforehand with our exhaustive L2 search we were returning 7460, 10940, 3781, and 5747. Now, we see a slightly different order to our results - and two different vectors, 5013 and 5370.

Each of our speed optimization operations, IVF and PQ, come at the cost of accuracy. Now, if we print out these results we will nonetheless find that each item is still a relevant match:

In [156]:
[f'{i}: {sentences[i]}' for i in I[0]]


['2332: Different football players are teaming on a field',
 '2711: A football player kicks the ball.',
 '2902: Different teams are playing football on a field',
 '3193: Two groups of people are playing football']